<a href="https://colab.research.google.com/github/nathasya-abenita/MA2151-Simulation-and-Mathematical-Computation./blob/main/Pit_Viper_and_Rodent_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Project**
MA2151 Simulation and Mathematical Computation \\

The objective of this project is to simulate the movement of a pit viper preying a rodent. A pit viper is able to perceive heat, thus it preys rodent by moving to area with warmer temperature. A rodent has a stable temperature around 37 degree celcius, thus its heat will warm its surroundings. The simulation uses
* agent-based modelling to model the movement of viper and rodent 
* heat diffusion model for the rodent's heat diffusion to the envinronment.

# Case 1 (Rodent walks randomly)
In this program, environment matrices from the simulation is saved in matrix *matDisplay*. \\

Information on used variables:
*  ` mat[i,j,0] ` : Status matrix
*  ` mat[i,j,1] ` : Temperature matrix
* Status code: -1 (border), 0 (empty), 1 (viper), 2 (rodent)
* Viper agent info: `agents[0,0]` (row), `agents[0,1]` (column)
* Rodent agent info: `agents[1,0]` (row), `agents[1,1]` (column), `agents[1,2]` (temperature)

Assumption that is used:
* Absorbing border condition
* Moore neighborhood
* Environment matrix size: 25x25
* Heat diffusion parameter: 0.1



In [1]:
# PIT VIPER AND RODENT SIMULATION
# Input (Initialized parameter): m (row), n (column), heat diffusion parameter, environment temperature
# Output: Environment matrices from every iteration (will be animated later)
import numpy as np

# Environment Initialization Function
def InitializeEnvironment (m, n, ENV_TEMP):
  # Main environment matrix is initialized with value 0 (Status code for empty)
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0]: matStatus, mat[i,j,1]: matTemperature
  # Setting boundary's status to border (-1) and assigning environment temperature to all cells
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1: # Untuk sel border
        mat[i][j][0] = -1
        mat[i][j][1] = ENV_TEMP # Absorbing border condition is used
      else:
        mat[i][j][1] = ENV_TEMP 
  # Agent initialization (viper and rodent)
  agents = []
  # Viper is assigned to a random position in the environment [row, column]
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Rodent is assigned to a random position in the environment and random temperature around 37 degree celcius [row, column, temperature]
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  # Setting both agents' position in the environment matrix
  mat[agents[0][0]][agents[0][1]][0] = 1 # status code for viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # status code for rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # Temperature in rodent's position is set to rodent's tempreature
  return mat, agents

# Function to update environment matrix status based on agent's movement
def UpdateStatus (mat, agents):
  # Rodent's movement is done first before viper's movement
  mat, agents = RodentWalk(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

# Fungsi for rodent's movement
def RodentWalk (mat, agents):
  # Collecting row and column position of the rodent
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  # Array initialization for empty neighbor cells
  movementChoice = []
  # Moore neighbor hood is used to check empty cells
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: # if the status is zero (empty)
      movementChoice.append([row+x, column+y])
  # Appending the possibility that the rodent doesn't move
  movementChoice.append([row, column]) 
  # Randomly choosing the movement choice
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  # Rodent's new position is assigned to the environment matrix and agents' array
  if newPosition[0] == row and newPosition[1] == column: 
    pass 
  else: 
    newRow, newColumn = newPosition[0], newPosition[1]
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

# Function for viper's movement
def ViperWalk (mat, agents):
  # Collecting viper's current position
  row, column = agents[0][0], agents[0][1]
  # Array for empty neighbor cell or containing rodent, its temperature, and final movement choice
  emptyNeighbors, emptyNeighborsT, movementChoice = [], [], []
  # Collecting empty or containing rodent neighbor cells
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0:
      emptyNeighbors.append([row+x, column+y])
  # Temperature for corresponding cells is collected
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  # Choosing maximum temperature
  maxT = max(emptyNeighborsT)
  # Cell with maximum temperature is chosen as possible final movements
  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  # Final movement is randomly chosen from possible ones
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  # Environment matrix and agent's matrix are updated based on viper's movement
  newRow, newColumn = newPosition[0], newPosition[1]
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

# Function for heat diffusion
def HeatDiffusion (m, n, mat, DIFF_RATE):
  # Diffusion is done in the interior cells
  for i in range (1, m+1):
    for j in range (1, n+1):
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      mat[i][j][1] = (1 - 8*DIFF_RATE)* mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

# Function to check whether both agents' position is the same (rodent is preyed)
def CheckPrey(viper, rodent):
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr: 
    return True 
  else:
    return False

# Main Algorithm
m, n, ENV_TEMP, DIFF_RATE = 25, 25, 30, 0.1 
matDisplay = []
# Algorithm is run by looping process of phase 0-1-2 back to 1-2 until rodent is preyed
phase = 0
while phase != 3:
  if phase == 0:
    mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
    phase += 1
  elif phase == 1:
    mat = HeatDiffusion(m, n, mat, DIFF_RATE)
    phase += 1
    matDisplay.append(mat)
  elif phase == 2:
    mat, agents = UpdateStatus(mat, agents)
    print(agents)
    if CheckPrey(agents[0], agents[1]):
      phase += 1
    else:
      phase = 1

[[20, 9], [18, 19, 36.650423931945674]]
[[21, 9], [19, 20, 36.652605593211284]]
[[22, 10], [19, 19, 36.82348894300533]]
[[21, 11], [18, 19, 36.59574024136491]]
[[20, 12], [17, 19, 36.511919353716735]]
[[19, 13], [17, 19, 36.511919353716735]]
[[18, 14], [18, 18, 37.17859732546473]]
[[18, 15], [19, 17, 37.36786385153558]]
[[18, 16], [18, 16, 36.68000452091142]]


# Animation for Case 1
In addition to the previous program, lines of code to animate *matDisplay* is added.

In [6]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation


def InitializeEnvironment (m, n, ENV_TEMP):
  # Initialize main matrix
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0] is matStatus ; mat[i,j,1] is matTemp
  # Setting border status and environment temperature
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1:
        mat[i][j][0] = -1 # border
        mat[i][j][1] = ENV_TEMP # absorbing border condition
      else:
        mat[i][j][1] = ENV_TEMP
  # Initialize agents (viper and rodent)
  agents = []
  # Viper (row, column)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Rodent (row, column, temperature)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  print(agents)
  # Setting agents in the matrix
  mat[agents[0][0]][agents[0][1]][0] = 1 # status viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # status rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # temp in rodent's position to its temp
  return mat, agents

def UpdateStatus (mat, agents):
  mat, agents = RodentWalk(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

def RodentWalk (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: # if status in that cell is empty
      movementChoice.append([row+x, column+y])
  movementChoice.append([row, column]) # adding resting probability
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  if newPosition[0] == row and newPosition[1] == column:
    pass
  else:
    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

def ViperWalk (mat, agents):
  row, column = agents[0][0], agents[0][1]
  emptyNeighbors = []
  emptyNeighborsT = []
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0: # if status in that cell is empty or contains rodent
      emptyNeighbors.append([row+x, column+y])
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  maxT = max(emptyNeighborsT)

  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  
  newRow, newColumn = newPosition[0], newPosition[1]
  # Update matrix to viper's new position
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  # Update viper's information
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

def HeatDiffusion (m, n, mat, DIFF_RATE):
  for i in range (1, m+1):
    for j in range (1, n+1):
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      mat[i][j][1] = (1 - 8*DIFF_RATE)* mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

def CheckPrey(viper, rodent):
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr:
    return True
  else:
    return False

# Main Program

m, n, ENV_TEMP, DIFF_RATE = 25, 25, 30, 0.1
matDisplay = []
phase = 0
while phase != 3:
  if phase == 0:
    mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
    phase += 1
  elif phase == 1:
    mat = HeatDiffusion(m, n, mat, DIFF_RATE)
    phase += 1

    matStatus =np.zeros((m+2,n+2))
    for i in range(m+2):
      for j in range(m+2):
        matStatus[i][j] = mat[i][j][1]
        if i  == agents[0][0] and j== agents[0][1]:     # viper
          matStatus[i][j] = 35
        elif i  == agents[1][0] and j== agents[1][1]:       # rodent
          matStatus[i][j] = 35

    matDisplay.append(matStatus)

  elif phase == 2:
    mat, agents = UpdateStatus(mat, agents)
    if CheckPrey(agents[0], agents[1]): # Checking whether agents' position are the same (rodent is preyed)
      phase += 1
    else:
      phase = 1

# proceed to display all matrices in matDisplay

# Setting figure, heatmap
fig = plt.figure()

def init():
  plt.clf()
  return None

def animate(i):
  plt.clf()
  ax = sns.heatmap(matDisplay[i-1],
                   cmap="magma",
                   square=True,
                   xticklabels=False,
                   yticklabels=False,
                   )

  return None

anim = animation.FuncAnimation(fig,
                               animate,
                               frames=range(1, len(matDisplay)+1, 1),
                               blit=False,
                               interval=250,
                               init_func=init)

from matplotlib import rc
from IPython.display import HTML
rc('animation', html='jshtml')
anim

[[19, 10], [10, 15, 37.030391188975976]]


<Figure size 432x288 with 0 Axes>

# Average Iteration for Case 1
Addition lines of code is added to run simulation for 100 times and the average movement that is needed for the viper to prey rodent is calculated.

In [7]:
import numpy as np
def InitializeEnvironment (m, n, ENV_TEMP):
  # Initialize main matrix
  mat = np.zeros((m+2, n+2, 2)) # mat[i,j,0] is matStatus ; mat[i,j,1] is matTemp
  # Setting border status and environment temperature
  for i in range (m+2):
    for j in range (n+2):
      if i == 0 or j == 0 or i == m+1 or j == n+1:
        mat[i][j][0] = -1 # border
        mat[i][j][1] = ENV_TEMP # absorbing border condition
      else:
        mat[i][j][1] = ENV_TEMP
  # Initialize agents (viper and rodent)
  agents = []
  # Viper (row, column)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1)]) 
  # Rodent (row, column, temperature)
  agents.append([np.random.randint(1, m+1), np.random.randint(1, n+1), np.random.uniform(36.5,37.5)])
  # Setting agents in the matrix
  mat[agents[0][0]][agents[0][1]][0] = 1 # status viper
  mat[agents[1][0]][agents[1][1]][0] = 2 # status rodent
  mat[agents[1][0]][agents[1][1]][1] = agents[1][2] # temp in rodent's position to its temp
  return mat, agents

def UpdateStatus (mat, agents):
  mat, agents = RodentWalk1(mat, agents)
  mat, agents = ViperWalk(mat, agents)
  return mat, agents

def RodentWalk1 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2]
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] == 0: # if status in that cell is empty
      movementChoice.append([row+x, column+y])
  movementChoice.append([row, column]) # adding resting probability
  newPositionId = np.random.randint(0, len(movementChoice))
  newPosition = movementChoice[newPositionId]
  if newPosition[0] == row and newPosition[1] == column:
    pass
  else:
    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
  return mat, agents

def RodentWalk2 (mat, agents):
  row, column, temp = agents[1][0], agents[1][1], agents[1][2] # Rodent's position
  # Checking whether there's a snake nearby or not
  isNearViper = False
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if isNearViper:
      break
    if mat[row+x][column+y][0] == 1: # if status in that cell contains viper
      if np.random.uniform() < 0.9: # 90% chance that rodent will run twice as fast  
        isNearViper = True
  # Executing movement 
  if not isNearViper: # Normal walk
    return RodentWalk1(m, n, mat, agents)
  else: # Twice as fast as viper
    rowv, colv = agents[0][0], agents[0][1] # Viper's position
    emptyNeighbors, emptyNeighborsDist, movementChoice = [], [], []
    # Checking empty cells
    for x,y in [[-2,2],[0,2],[2,2],[2,0],[2,-2],[0,-2],[-2,-2],[-2,0]]:
      if 0 <= row+x <= m+1 and 0 <= column+y <= n+1:
        if mat[row+x][column+y][0] == 0: # if status in that cell is empty
          emptyNeighbors.append([row+x, column+y])
    # Choosing the furthest cells from viper
    for x,y in emptyNeighbors:
      emptyNeighborsDist.append(abs(x-rowv)**2 + abs(y-colv)**2)
    maxDist = max(emptyNeighborsDist)

    for i in range (len(emptyNeighbors)):
      if emptyNeighborsDist[i] == maxDist:
        movementChoice.append(emptyNeighbors[i])
  
    if len(movementChoice) == 1:
      newPosition = movementChoice[0]
    else:
      newPositionId = np.random.randint(0, len(movementChoice))
      newPosition = movementChoice[newPositionId]

    newRow, newColumn = newPosition[0], newPosition[1]
    # Update matrix
    mat[newRow][newColumn][0] = 2
    mat[row][column][0] = 0
    mat[newRow][newColumn][1] = temp
    # Update rodent's information
    agents[1][0] = newRow
    agents[1][1] = newColumn
    agents[1][2] = np.random.uniform(36.5,37.5)
    return mat, agents

def ViperWalk (mat, agents):
  row, column = agents[0][0], agents[0][1]
  emptyNeighbors = []
  emptyNeighborsT = []
  movementChoice = []
  for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
    if mat[row+x][column+y][0] >= 0: # if status in that cell is empty or contains rodent
      emptyNeighbors.append([row+x, column+y])
  for x,y in emptyNeighbors:
    emptyNeighborsT.append(mat[x][y][1])
  maxT = max(emptyNeighborsT)

  for i in range (len(emptyNeighbors)):
    if emptyNeighborsT[i] == maxT:
      movementChoice.append(emptyNeighbors[i])
  
  if len(movementChoice) == 1:
    newPosition = movementChoice[0]
  else:
    newPositionId = np.random.randint(0, len(movementChoice))
    newPosition = movementChoice[newPositionId]
  
  newRow, newColumn = newPosition[0], newPosition[1]
  # Update matrix to viper's new position
  mat[newRow][newColumn][0] = 1
  mat[row][column][0] = 0
  # Update viper's information
  agents[0][0] = newRow
  agents[0][1] = newColumn
  return mat, agents

def HeatDiffusion (m, n, mat, DIFF_RATE):
  for i in range (1, m+1):
    for j in range (1, n+1):
      sumNeighbor = 0
      for x,y in [[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0]]:
        sumNeighbor += mat[i+x][j+y][1]
      mat[i][j][1] = (1 - 8*DIFF_RATE) * mat[i][j][1] + DIFF_RATE * sumNeighbor
  return mat

def CheckPrey(viper, rodent):
  rowv, colv, rowr, colr = viper[0], viper[1], rodent[0], rodent[1]
  if rowv == rowr and colv == colr:
    return True
  else:
    return False

# Main Program
matLength = []
for k in range(100):
  m, n, ENV_TEMP, DIFF_RATE = 25, 25, 25, 0.1
  matDisplay = []
  phase = 0
  while phase != 3:
    if phase == 0:
      mat, agents = InitializeEnvironment(m, n, ENV_TEMP)
      phase += 1
    elif phase == 1:
      mat = HeatDiffusion(m, n, mat, DIFF_RATE)
      phase += 1
      matDisplay.append(mat)
    elif phase == 2:
      print
      mat, agents = UpdateStatus(mat, agents)
      if CheckPrey(agents[0], agents[1]): # Checking whether agents' position are the same (rodent is preyed)
        phase += 1
      else:
        phase = 1
  matLength.append(len(matDisplay))

average = sum(matLength)/len(matLength)
print("Average iteration or agent's movement:", average)

Average iteration or agent's movement: 16.65


# **Remark**
There is a continuation of the simulation, that is when rodent can detect viper when its near rodent (placed in neighbor cells). After detecting viper's presence, rodent will be able to move two times further than its usual reach. The rest of the program will be updated in future time. \\

A detail discussion of the simulation can be found in the video below that is made in Bahasa Indonesia.

[Discussion Video](https://youtu.be/GAPxBPRUEVU)
